In [2]:
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, eq, Z1, pairing, neg, FQ, final_exponentiate, FQ12, is_inf
import numpy as np
import galois 
from functools import reduce
GF = galois.GF(curve_order)

In [9]:
x = GF(5)
y = GF(10)

# Elliptic Curve Operations
def ecmul(point, scalar):
    if(is_inf(point) or scalar == 0):
        return Z1
    return multiply(point, mod_scalar(int(scalar)))

def ecadd(s, *args):
    if(s == Z1) and not args:
        return Z1
    
    for x in args:
        if x == Z1:
            continue #skip adding Z1
        if isinstance(x, tuple) and len(x) == 2:
            # Directly add if x is a point on the curve
            s = add(s, x)
        else:
            # Assume x is a scalar and try multiplying with G1
            try:
                s = add(s, ecmul(G1, x))
            except Exception:
                # If it fails, try with G2
                s = add(s, ecmul(G2, x))
    return s

# Modulo Operations
def mod_scalar(a):
    return int(a % curve_order)

def mod_array(a):
    return np.array([mod_scalar(u) for u in a.flatten()]).reshape(a.shape)

# R1CS
L = np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,-5,0,0,0,0,0]
    ])

R = np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,1,0]
    ])

O = np.array([
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1],
    [-2,1,0,3,0,-1,0,-2]
    ])

r1cs_m, r1cs_n  = L.shape[1], L.shape[0] #m columns, n rows

v1 = x * x
v2 = v1 * x
v3 = y * y
v4 = v2 * y
out = v2 + GF(2) * v4 - GF(5) * x * v3 - GF(3) * y + GF(2)

witness = GF(np.array([GF(1),out,x,y,v1,v2,v3,v4]))
result = O.dot(witness) == np.multiply(L.dot(witness),R.dot(witness))
assert result.all(), "result contains an inequality"

# Polynomial Interpolation, Finite Field Arithmetic
L_galois, R_galois, O_galois = GF(mod_array(L)), GF(mod_array(R)), GF(mod_array(O))
tx_roots = GF([1,2,3,4,5])
tx_poly = galois.Poly(np.flip(np.polynomial.polynomial.polyfromroots(tx_roots)).astype(int),  field=GF)

# Polynomial Functions
def interpolate_columns(col):
    return galois.lagrange_poly(tx_roots, col)

def matrix_to_polys(matrix):
    return np.apply_along_axis(interpolate_columns, 0, matrix)

def print_polys(name, polys):
    print(name, "polys:")
    for poly in polys:
        print("  ", poly)

def mul_polys_by_witness(polys, witness):
    return [poly * int(witness[i]) for i, poly in enumerate(polys)]

def reduce_polys_to_one(polys, start_index, end_index):
    return reduce(lambda a, b: a + b, polys[start_index:end_index])

# R1CS to QAP
U_polys, V_polys, W_polys = matrix_to_polys(L_galois), matrix_to_polys(R_galois), matrix_to_polys(O_galois)
Ua_polys, Va_polys, Wa_polys = mul_polys_by_witness(U_polys, witness), mul_polys_by_witness(V_polys, witness), mul_polys_by_witness(W_polys, witness)
Ua_poly, Va_poly, Wa_poly = reduce_polys_to_one(Ua_polys, 0, r1cs_m), reduce_polys_to_one(Va_polys, 0, r1cs_m), reduce_polys_to_one(Wa_polys, 0, r1cs_m)

hx_poly, remainder = divmod(((Ua_poly * Va_poly) - Wa_poly), tx_poly)
assert remainder == 0, "remainder is not zero"
hx_tx_poly = hx_poly * tx_poly

# Trusted Setup
def setup_trusted_parameters():
    tau, alpha, beta, gamma, delta = [GF(i) for i in (9, 10, 15, 21, 5)]
    gamma_inv, delta_inv = [pow(int(x), -1, curve_order) for x in (gamma, delta)]
    gamma_invg1, delta_invg1 = ecmul(G1, int(gamma_inv)), ecmul(G1, int(delta_inv))
    return tau, alpha, beta, gamma, delta, gamma_inv, delta_inv, gamma_invg1, delta_invg1

tau, alpha, beta, gamma, delta, gamma_inv, delta_inv, gamma_invg1, delta_invg1 = setup_trusted_parameters()


alpha_times_delta_inv1 = ecmul(G1, int(alpha * delta_inv))
beta_times_delta_inv1 = ecmul(G1, int(beta * delta_inv))
alpha_times_gamma_inv1 = ecmul(G1, int(alpha * gamma_inv))
beta_times_gamma_inv1 = ecmul(G1, int(beta * gamma_inv))

alphag1 = ecmul(G1, alpha)
betag1 = ecmul(G1, beta)
betag2 = ecmul(G2, beta)
deltag1 = ecmul(G1, delta)
deltag2 = ecmul(G2, delta)
gammag1 = ecmul(G1, gamma)
gammag2 = ecmul(G2, gamma)

public_witness_length = 2

r = GF(3)
s = GF(4)

# evaluate polynomials at tau
ua_tau, va_tau, wa_tau = int(Ua_poly(tau)), int(Va_poly(tau)), int(Wa_poly(tau))
hx_tx_poly_tau = int(hx_tx_poly(tau))

# split polynomials into public and private parts
ua_private_poly = reduce_polys_to_one(Ua_polys, public_witness_length, r1cs_m)
va_private_poly = reduce_polys_to_one(Va_polys, public_witness_length, r1cs_m)
wa_private_poly = reduce_polys_to_one(Wa_polys, public_witness_length, r1cs_m)

ua_public_poly = reduce_polys_to_one(Ua_polys, 0, public_witness_length)
va_public_poly = reduce_polys_to_one(Va_polys, 0, public_witness_length)
wa_public_poly = reduce_polys_to_one(Wa_polys, 0, public_witness_length)

ua_private, va_private, wa_private = int(ua_private_poly(tau)), int(va_private_poly(tau)), int(wa_private_poly(tau))
ua_public, va_public, wa_public = int(ua_public_poly(tau)), int(va_public_poly(tau)), int(wa_public_poly(tau)) 


# Generate Proofs, Verify Pairings
A1 = ecadd(alphag1, ua_tau, ecmul(deltag1,r))
B2 = ecadd(betag2, va_tau, ecmul(deltag2, s))
B1 = ecadd(betag1, va_tau, ecmul(deltag1, s))
C1_private = ecadd(
    ecmul(beta_times_delta_inv1, ua_private), 
    ecmul(alpha_times_delta_inv1, va_private),
    ecmul(delta_invg1, wa_private),
    ecmul(delta_invg1, hx_tx_poly_tau), 
    ecmul(A1, int(s)), 
    ecmul(B1, int(r)), 
    ecmul(deltag1, int(-r * s))
)

C1_public = ecadd(
    ecmul(beta_times_gamma_inv1, ua_public),
    ecmul(alpha_times_gamma_inv1, va_public),
    ecmul(gamma_invg1, wa_public),
)

AB = pairing(B2,neg(A1))
ab = pairing(betag2,alphag1)
CD = pairing(gammag2, C1_public)
EF = pairing(deltag2, C1_private)

# Coeffs AB = Coeffs ab * CD * EF
final_exponentiate(AB * ab * CD * EF) == FQ12.one()

True

In [28]:
# Trusted Setup for encrypted G16
def powers_of_tau(tau, n, G):
    return [ecmul(G, int(tau) ** i) for i in range(n)]

def generate_ptau_for_c(U_polys, V_polys, W_polys, beta, alpha, tau):
    return [ecmul(G1, int(beta * U_polys[i](tau) + alpha * V_polys[i](tau) + W_polys[i](tau))) for i in range(0, len(Ua_polys))]

def inner_product(powers_of_tau, poly_coeffs):
    return reduce(ecadd, (ecmul(point, int(coeff)) for point, coeff in zip(powers_of_tau, poly_coeffs)), Z1)

p_taug1 = powers_of_tau(tau, r1cs_m, G1)
p_taug2 = powers_of_tau(tau, r1cs_n, G2)
tx_p_taug1 = powers_of_tau(tau, tx_poly.degree, G1)
tx_tau = tx_poly(int(tau))
tx_p_taug1_tau = [ecmul(tx_p_taug1[i], int(tx_tau)) for i in range(0, len(tx_p_taug1))]
c_p_taug1 = generate_ptau_for_c(U_polys, V_polys, W_polys, beta, alpha, tau)

Ua_taug1 = inner_product(p_taug1, Ua_poly.coeffs[::-1])
Va_taug2 = inner_product(p_taug2, Va_poly.coeffs[::-1])
Wa_taug1 = inner_product(p_taug1, Wa_poly.coeffs[::-1])
hx_tx_taug1 = inner_product(tx_p_taug1_tau, hx_poly.coeffs[::-1])

# test encrypted pairing Ua * Va = Wa * hx_tx
# final_exponentiate(
#     pairing(Va_taug2, neg(Ua_taug1)) * 
#     pairing(G2, ecadd(Wa_taug1, hx_tx_taug1))) == FQ12.one()

# test encrypted p(a + A, b + B) = p(a, b) + p(G2, C)
ca_taug1 = inner_product(c_p_taug1, witness)

final_exponentiate(
    pairing(ecadd(betag2, Va_taug2),neg(ecadd(alphag1,Ua_taug1))) * 
    pairing(betag2, alphag1) * 
    pairing(G2, ecadd(ecadd(ca_taug1, hx_tx_taug1))) == FQ12.one()
)




# use powers of tau to encrypt the polynomials with witness
# Ua_poly, Va_poly, Wa_poly, tx_poly
# print("Ua_poly", Ua_poly)

1